In [1]:
from random import randint, random
import collections
import os

import numpy as np
import math
import time
from collections import deque
import torch as T
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import sklearn.neighbors
import sklearn.metrics
import sklearn.mixture
import networkx.algorithms.community as nx_comm
from scipy import stats
import networkx as nx
import matplotlib.pyplot as plt
import PIL
from nvidia.dali.plugin.pytorch import DALIGenericIterator

from imageComponent import *
from remote_read_write import scp_read_wrapper, scp_write_wrapper
scp_Tload = scp_read_wrapper(T.load, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')
scp_Tsave = scp_write_wrapper(T.save, 'ly_1112103017@172.16.35.121', 30907, '/public/ly/zyz/cluster_id_rsa', '/public/ly/zyz/imageCL/tmp')

dali_device = 1
device = 'cuda:3'
T.backends.cudnn.benchmark = True
T.backends.cudnn.deterministic = True
use_amp = False

/public/ly/anaconda3/envs/zyz/lib/python3.10/site-packages/nvidia/dali/backend.py:46: Warning: DALI support for Python 3.10 is experimental and some functionalities may not work.
  deprecation_warning("DALI support for Python 3.10 is experimental and some functionalities "


In [2]:
model = ViTCifar()
if_normalize = True
root = '/public/home/ly_1112103017/zyz/imageCL/ckpt/CLBadCifarViT2'
files = (0, 1201, 10)

In [3]:
trainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image3/train'
rawtrainfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image3/train'
valfolder = '/public/ly/zyz/imageCL/dataset/cifar10-image3/test'
size = (32, 32)
batch_size = 3125

In [4]:
rawtrainpipe = DALIValdImageFolders(root=rawtrainfolder, size=size, batch_size=batch_size, num_threads=6, device_id=dali_device)
rawtrainloader = dali.plugin.pytorch.DALIGenericIterator([rawtrainpipe], reader_name ='reader', output_map=['image', 'label'],
                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)
valpipe = DALIValdImageFolders(root=valfolder, size=size, batch_size=batch_size, num_threads=4, device_id=dali_device)
valloader = dali.plugin.pytorch.DALIGenericIterator([valpipe], reader_name='reader', output_map=['image', 'label'],
                                                    last_batch_policy=dali.plugin.base_iterator.LastBatchPolicy.PARTIAL)

In [ ]:
#CH, 1 0.2 0.1 RLD, linear tacc vacc, GCN tacc vacc
recorder = np.zeros((files[1], 10))
for i in range(*files):
    t = time.time()
    #while not os.path.exists('%s/%04d.pth' %(root, i)):
    #    time.sleep(100)
    model.load_state_dict(scp_Tload('%s/%04d.pth' %(root, i), map_location=device))
    model = model.to(device, memory_format = T.channels_last)
    with T.no_grad():
        tX, ty = extract_feature(rawtrainloader, model, device, normalize=if_normalize, use_amp=use_amp)
        vX, vy = extract_feature(valloader, model, device, normalize=if_normalize, use_amp=use_amp)

        recorder[i, 0] = calinski_harabasz_score(tX, ty).item()
        recorder[i, 1:4] = [k.detach().cpu().numpy() for k in relative_local_density_score(tX, ty, [1, 0.2, 0.1])]
    lh = NNHead(nn.Linear, tolerate=30, label_smoothing=0, lr=1e-1, use_amp=use_amp, verbose=0).fit(tX, ty, vX, vy)
    recorder[i, 4:6] = lh.best_acc, lh.best_val_acc
    gh = GraphHead(0.2, mask_p=[0.05, 0.5], tolerate=50, label_smoothing=0, lr=1e-1, use_amp=use_amp, verbose=0).fit(tX, ty, vX, vy)
    recorder[i, 6:8] = gh.best_acc, gh.best_val_acc
    del gh
    gh = GraphHead(0.1, mask_p=[0.05, 0.5], tolerate=50, label_smoothing=0, lr=1e-1, use_amp=use_amp, verbose=0).fit(tX, ty, vX, vy)
    recorder[i, 8:10] = gh.best_acc, gh.best_val_acc
    #del gh
    del tX, ty, vX, vy, lh, gh
    T.cuda.empty_cache()
    print(i)
    print(time.time() - t)
    scp_Tsave(recorder, '%s/full_recorder.pth' %root)
    print(recorder[i], end='\n\n')

0
194.96734380722046
[5.92507935e+02 1.37086902e-02 7.44909868e-02 1.25474349e-01
 4.48460000e-01 4.30400000e-01 3.20871503e-01 3.12600000e-01
 3.54394904e-01 3.51600000e-01]



probe = nn.Linear(512, 10)
probe = probe.to(device, memory_format=T.channels_last).train()
encoder = model
encoder.load_state_dict(T.load('%s/%04d.pth' %(root, 1256)))
#jitencoder = T.jit.script(encoder).to(device, memory_format = T.channels_last)
jitencoder = encoder.to(device, memory_format = T.channels_last)
r, probe = feature_eval(jitencoder, 512, trainloader=trainloader, rawtrainloader=rawtrainloader, valloader=valloader,
                        num_class=10, batch_size=batch_size, tolerate=3, device=device, init_probe=probe,
                        normalize=True, use_amp=True, verbose=True)